In [57]:
import pandas as pd
import numpy as np

In [58]:
FILEPATH = "/Volumes/OFFCAMPUS/Jenn/Imported data/cleaned/"
participants = ["105", "106", "107", "904", "905", "906"]

dfs = [pd.read_csv("%s%s_cleaned_ml.csv" % (FILEPATH, s)) for s in participants]

df = pd.concat(dfs, axis=0, ignore_index=True)

In [96]:
df

,Time,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,E56,E57,E58,E59,E60,E61,E62,E63,E64,E65
0,-200.0,13.649351,4.253679,9.996485,9.481434,18.026508,5.564005,12.778870,1.174925,13.478706,...,-2.362542,-0.461481,-9.263395,-2.207499,3.446589,-3.198227,-3.198227,-3.198227,-3.198227,-3.198227
1,-199.0,13.828758,4.976443,10.084705,9.582227,18.480547,5.537510,12.838730,1.211721,13.147108,...,-3.272416,-0.463592,-10.913692,-2.212922,3.910747,-3.019854,-3.019854,-3.019854,-3.019854,-3.019854
2,-198.0,14.142060,5.850959,10.178824,9.686580,19.001780,5.393631,12.747041,1.206468,12.338480,...,-4.002994,-0.402802,-11.890976,-2.120070,4.458347,-2.751333,-2.751333,-2.751333,-2.751333,-2.751333
3,-197.0,14.560205,6.793514,10.284312,9.803214,19.552628,5.139943,12.524466,1.158097,11.103312,...,-4.511529,-0.304838,-12.029873,-1.959178,5.037091,-2.393907,-2.393907,-2.393907,-2.393907,-2.393907
4,-196.0,15.053234,7.721226,10.407560,9.938207,20.092878,4.792314,12.198002,1.070401,9.519570,...,-4.765975,-0.191731,-11.221484,-1.756361,5.597899,-1.954948,-1.954948,-1.954948,-1.954948,-1.954948
5,-195.0,15.593115,8.557594,10.554512,10.093856,20.581530,4.373208,11.798501,0.950353,7.686402,...,-4.747547,-0.080520,-9.425688,-1.532278,6.097967,-1.447423,-1.447423,-1.447423,-1.447423,-1.447423
6,-194.0,16.155512,9.237585,10.729192,10.267609,20.977789,3.909213,11.357679,0.805719,5.716181,...,-4.452627,0.017718,-6.677195,-1.301555,6.503740,-0.889459,-0.889459,-0.889459,-0.889459,-0.889459
7,-193.0,16.721960,9.712637,10.932883,10.451797,21.243439,3.428840,10.905655,0.644200,3.726250,...,-3.893324,0.098082,-3.084711,-1.072368,6.792570,-0.303469,-0.303469,-0.303469,-0.303469,-0.303469
8,-192.0,17.279751,9.952982,11.162840,10.633914,21.344727,2.960049,10.468356,0.472011,1.831099,...,-3.096823,0.161356,1.176197,-0.847298,6.953444,0.285073,0.285073,0.285073,0.285073,0.285073
9,-191.0,17.820412,9.949510,11.411726,10.796390,21.254143,2.527737,10.065152,0.293260,0.134708,...,-2.103785,0.213223,5.875682,-0.624043,6.987071,0.848734,0.848734,0.848734,0.848734,0.848734


In [59]:
# samples are df rows mod 1200, pre is 0-199, post is 200-1199

# extract pre rows and post rows, randomly choose 200-400ms as post
pre = df[df.Time < 0]
post = df[df.Time >= 200][df.Time < 400]

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [60]:
pre = pre.drop(columns=["Time", "E65"], axis=1)
post = post.drop(columns=["Time", "E65"], axis=1)

In [61]:
timeFrame = 200

preX = pre.values
(i,j) = preX.shape
preX = np.reshape(preX, (i // 200, j * 200))

In [62]:
postX = post.values
(i,j) = postX.shape
postX = np.reshape(postX, (i // 200, j * 200))

In [63]:
(preN, preD) = preX.shape
(postN, postD) = postX.shape

In [64]:
X = np.concatenate((preX, postX), axis=0)
(n, d) = X.shape
assert(n == preN + postN)
assert(d == postD)
assert(d == preD)

In [65]:
y = np.concatenate((np.zeros(preN), np.ones(postN)))
assert(y.shape == (n,))

In [66]:
# just use this : on next one
# from sklearn.model_selection import train_test_split 

rand_indices = np.random.permutation(n)
split = int(n * 0.8)
train_idx, valid_idx = rand_indices[:split], rand_indices[split:]
train_X, valid_X, train_y, valid_y = X[train_idx, :], X[valid_idx, :], y[train_idx], y[valid_idx]

In [67]:
# from sklearn.svm import SVC
# model = SVC(gamma='auto', kernel='linear')
# model.fit(train_X, train_y)

In [87]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(train_X, train_y)

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [88]:
# from sklearn.neighbors import KNeighborsClassifier
# from dtw import DTW

# model = KNeighborsClassifier(metric=DTW)
# model.fit(train_X, train_y)

In [89]:
Xhat = model.predict(valid_X)

In [90]:
np.mean(Xhat != valid_y)

0.07734806629834254

In [91]:
Xhat

array([1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
       1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1.])

In [92]:
valid_y

array([1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
       1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.])

In [93]:
np.mean(model.predict(train_X) != train_y)

0.0027662517289073307

In [94]:
Xhat != valid_y

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True, False, False, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False,

In [95]:
from sklearn.metrics import classification_report
print(classification_report(valid_y, Xhat))

              precision    recall  f1-score   support

         0.0       0.90      0.96      0.93        98
         1.0       0.95      0.88      0.91        83

    accuracy                           0.92       181
   macro avg       0.93      0.92      0.92       181
weighted avg       0.92      0.92      0.92       181

